# Proyecto SQL

**- Datos:** 
books, authors, publishers, ratings, reviews. 

**- Objetivos del estudio:** 
1. Analizar la información disponible sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros.
2. Responder preguntas clave relacionadas con el rendimiento editorial, la participación de usuarios y la calidad percibida de los libros.
3. Utilizar consultas SQL para obtener métricas relevantes y derivar conclusiones basadas en datos, generando asi una propuesta de valor para un nuevo producto.

## 1. Conexión a la base de datos 

In [7]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## 2. Exploración de datos

In [11]:
pd.read_sql("SELECT * FROM books LIMIT 5;", engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [12]:
pd.read_sql("SELECT * FROM authors LIMIT 5;", engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [13]:
pd.read_sql("SELECT * FROM publishers LIMIT 5;", engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [14]:
pd.read_sql("SELECT * FROM ratings LIMIT 5;", engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
pd.read_sql("SELECT * FROM reviews LIMIT 5;", engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Observaciones:** Según la información de las primeras líneas de los datos y el diagrama proporcionado, vemos que todas nuestras bases de datos pueden relacionarse entre si con alguna de sus columnas como por ejemplo, book_id, publisher_id y author_id; siendo books nuestra base más completa y con mas columnas relacionadas.

## 3. Consultas SQL 

### 3.1. Encuentra el número de libros publicados después del 1 de enero de 2000

In [22]:
query = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""

pd.read_sql(query, engine)

,books_after_2000
0,819


### 3.2. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro

In [32]:
query = """
SELECT
  b.book_id,
  b.title, 
  COUNT(r.rating_id)   AS total_ratings,
  AVG(r.rating)        AS avg_rating
FROM books b
JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title
ORDER BY total_ratings DESC, avg_rating DESC
LIMIT 20;
"""
pd.read_sql(query, engine)

,book_id,title,total_ratings,avg_rating
0,948,Twilight (Twilight #1),160,3.662500
1,750,The Hobbit or There and Back Again,88,4.125000
2,673,The Catcher in the Rye,86,3.825581
3,75,Angels & Demons (Robert Langdon #1),84,3.678571
4,302,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.414634
5,299,Harry Potter and the Chamber of Secrets (Harry...,80,4.287500
6,301,Harry Potter and the Order of the Phoenix (Har...,75,4.186667
7,722,The Fellowship of the Ring (The Lord of the Ri...,74,4.391892
8,79,Animal Farm,74,3.729730
9,300,Harry Potter and the Half-Blood Prince (Harry ...,73,4.246575


### 3.3.Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [25]:
query = """
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS count_books
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY count_books DESC
LIMIT 1;
"""
pd.read_sql(query, engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


### 3.4. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones

In [29]:
query = """
SELECT 
    a.author_id,
    a.author,
    AVG(sub.avg_rating) AS author_avg_rating
FROM authors a
JOIN (
    SELECT 
        b.author_id,
        b.book_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating_id) AS num_ratings
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.author_id, b.book_id
    HAVING COUNT(r.rating_id) >= 50
) sub ON a.author_id = sub.author_id
GROUP BY a.author_id, a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
"""
pd.read_sql(query, engine)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


### 3.5. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [30]:
query = """
WITH heavy_raters AS (
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
reviews_per_user AS (
    SELECT 
        r.username,
        COUNT(rv.review_id) AS count_reviews
    FROM heavy_raters r
    LEFT JOIN reviews rv ON r.username = rv.username
    GROUP BY r.username
)
SELECT AVG(count_reviews) AS avg_reviews_per_user
FROM reviews_per_user;
"""
pd.read_sql(query, engine)

,avg_reviews_per_user
0,24.333333


## Conclusiones
1. Tenemos un total de 819 libros publicados después del año 2000, esto nos permite identificar los libros más recientes. 
2. Vemos que tenemos valores muy variables en cuanto al total de valoraciones, teniendo en cuenta que Twilight tiene un total de ratings de casi el doble del libro que le sigue que es el Hobbit, lo cual quiere decir que es bastante popular y más confiable su media. Mientras que por el contrario hay libros con muy buen promedio de rating pero con apenas un par de valoraciones, lo cual hace la media poco confiable. 
Sin embargo, al separar los libros con mayor cantidad de valoraciones vemos que el promedio es similar moviendose entre 3,6 y 4,3, facilitando la identificación de los libros más populares y mejor percibidos por los usuarios. 
3. Teniendo en cuanta que las bases cuentan con otro tipo de materiales como folletos y publicaciones cortas, la editorial con mayor cantidad de libros como tal  publicados es Penguin Books con un total de 42 libros. 
4. Al analizar los libros con al menos 50 calificaciones, encontramos que los autores mejores valorados son J.K. Rowling y Mary GrandPré, autores de los libros de la saga Harry Potter. 
5. Los usuarios que califican más de 50 libros tienden a escribir en promedio 24 reseñas, mostrando un nivel elevado de participación a la hora de brindar su opinión para los demás usuarios interesados. 